## BANC CNS mirror landmarks
In this notebook, we will generate a "shortcut" to mirror BANC data. The general idea is:
1. Evenly sample points from one side of the CNS
2. Flip them to the other side to generate mirror-symmetrical source landmarks
3. Mirror those landmarks again but going via JRC2018M for the brain and via JRCVNC2018M for the VNC to generate the target landmarks

In [1]:
import numpy as np
import pandas as pd

import cloudvolume as cv

import flybrains
import navis

In [2]:
# Grab the whole brain mesh
if hasattr(flybrains.BANC, "_mesh_whole_brain"):
    del flybrains.BANC._mesh_whole_brain
mesh = flybrains.BANC.mesh_whole_brain

In [3]:
all_verts = mesh.vertices
bounds = np.vstack([all_verts.min(axis=0), all_verts.max(axis=0)]).astype(int)
bounds

array([[  79342,   35458,      43],
       [ 966128, 1131156,  315520]])

In [4]:
# Add 10 microns around the mesh
offset = 10000
bounds_off = bounds.astype(int)
bounds_off[0, :] -= offset
bounds_off[1, :] += offset
bounds_off

array([[  69342,   25458,   -9957],
       [ 976128, 1141156,  325520]])

In [5]:
# Sample points within the volume
res = 30000
sample = (
    np.mgrid[
        bounds_off[0, 0] : bounds_off[1, 0] : res,
        bounds_off[0, 1] : bounds_off[1, 1] : res,
        bounds_off[0, 2] : bounds_off[1, 2] : res,
    ]
    .reshape(3, -1)
    .T
)

# Drop points outside the volume
sample = sample[navis.in_volume(sample, mesh)]
sample.shape

(1217, 3)

In [15]:
# Next, we have to split the sampled points into:
# 1. the brain part which we will mirror via JRC2018M
# 2. the VNC part which we will mirror via JRCVNC2018M
is_brain = sample[:, 1] <= (90_000 * 4)
is_vnc = sample[:, 1] >= (120_000 * 4)

# Make sure no point is in both
(is_brain & is_vnc).sum()

np.int64(0)

In [20]:
# Next up: transforming the sample points
sample_mirr_brain = navis.mirror_brain(sample[is_brain], template="BANC", via="JRC2018F")
sample_mirr_vnc = navis.mirror_brain(sample[is_vnc], template="BANC", via="JRCVNC2018F")

In [ ]:
# This also looks decent
navis.plot3d([mesh, sample_mirr_brain, sample_mirr_vnc], scatter_kws=dict(color="r"))

In [27]:
# Just flipped
sample_flip_brain = navis.mirror_brain(sample[is_brain], template='BANC', warp=False)
sample_flip_vnc = navis.mirror_brain(sample[is_vnc], template='BANC', warp=False)

In [ ]:
# As expected this looks less then ideal
navis.plot3d([mesh, sample_flip_brain, sample_flip_vnc], scatter_kws=dict(color="r"))

In [29]:
# Bring it together
source = pd.DataFrame(np.vstack((sample_flip_brain, sample_flip_vnc)), columns=['x_flip', 'y_flip', 'z_flip']).round().astype(int)
target = pd.DataFrame(np.vstack((sample_mirr_brain, sample_mirr_vnc)), columns=['x_mirr', 'y_mirr', 'z_mirr']).round().astype(int)
lm = pd.concat((source, target), axis=1)
lm.head()

,x_flip,y_flip,z_flip,x_mirr,y_mirr,z_mirr
0,946128,115458,140043,879641,96112,137513
1,946128,145458,80043,894591,116458,69824
2,946128,145458,110043,889581,117106,101028
3,946128,145458,140043,877266,122310,130575
4,946128,175458,80043,891644,146115,65972


In [30]:
lm.to_csv('BANC_mirror_landmarks_nm.csv', index=False)